In [17]:
import numpy as np
import pandas as pd

from spotlight.datasets.movielens import get_movielens_dataset

dataset = get_movielens_dataset(variant='100K')

In [18]:
import torch

from spotlight.factorization.explicit import ExplicitFactorizationModel

model = ExplicitFactorizationModel(loss='regression',
                                   embedding_dim=128,  # latent dimensionality
                                   n_iter=10,  # number of epochs of training
                                   batch_size=1024,  # minibatch size
                                   l2=1e-9,  # strength of L2 regularization
                                   learning_rate=1e-3,
                                   use_cuda=torch.cuda.is_available())

In [19]:
from spotlight.cross_validation import random_train_test_split

train, test = random_train_test_split(dataset, random_state=np.random.RandomState(42))

print('Split into \n {} and \n {}.'.format(train, test))

Split into 
 <Interactions dataset (944 users x 1683 items x 80000 interactions)> and 
 <Interactions dataset (944 users x 1683 items x 20000 interactions)>.


In [20]:
model.fit(train, verbose=True)

Epoch 0: loss 13.121262345132948
Epoch 1: loss 7.378277739392051
Epoch 2: loss 1.769851405409318
Epoch 3: loss 1.0698453452013716
Epoch 4: loss 0.940627205975448
Epoch 5: loss 0.8858360874501965
Epoch 6: loss 0.8618532856808433
Epoch 7: loss 0.845095347754563
Epoch 8: loss 0.8312290522116649
Epoch 9: loss 0.8173502268670481


In [21]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(model, train)
test_rmse = rmse_score(model, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.889, test RMSE 0.936


In [46]:
dataset_path=".../ml-latest/movies.csv"

outdict={}
with open(dataset_path,'r') as fh:
    for line in fh:
        if line.startswith('mov'): continue
        pline=line.strip().split(',')
        movie_id, title =pline[0], pline[1]
        if movie_id in outdict:continue
        outdict[movie_id]=title
        
def recommend_movies(user_id, dataset, model, n_movies=5):
    
    ratings=model.predict(user_ids=user_id)
    indices=np.argpartition(ratings,-n_movies)[-n_movies:]
    best_movie_ids=indices[np.argsort(ratings[indices])]
    movie_id= [dataset[i] for i in best_movie_ids]

    return [outdict[str(v)] for v in list(movie_id)]

FileNotFoundError: [Errno 2] No such file or directory: '.../ml-latest/movies.csv'

In [47]:
recommend_movies(1, dataset.item_ids, model)

KeyError: '177'